# Extracting Human Faces From Images

## Imports

In [2]:
import cv2 
import numpy as np 
import os

from huggingface_hub import hf_hub_download
from PIL import Image
from supervision import Detections
from ultralytics.engine.results import Results
from ultralytics import YOLO

/home/pooh/miniconda3/envs/deep_learning/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Use Model

In [3]:
model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")
model = YOLO(model_path)

## Test

In [4]:
image_path = "/home/pooh/Downloads/IMG_20250418_233052.jpg"
original_image_bgr = cv2.imread(image_path)

In [ ]:
if original_image_bgr is None:
    print(f"Error: Could not load image from path: {image_path}")
    exit() 
else:
    print(f"Image loaded successfully: {image_path}")

    output: list[Results] = model(original_image_bgr) 
    results_ultralytics: Results = output[0]
    bboxes_xyxy = results_ultralytics.boxes.xyxy.cpu().numpy()

    output_crop_dir = "cropped_faces"
    os.makedirs(output_crop_dir, exist_ok=True) 

    base_filename = os.path.splitext(os.path.basename(image_path))[0]

    print(f"Found {len(bboxes_xyxy)} face(s). Cropping and saving...")

    if len(bboxes_xyxy) > 0:
        for i, bbox in enumerate(bboxes_xyxy):
            x_min, y_min, x_max, y_max = map(int, bbox[:4])

            padding = 0 
            h, w = original_image_bgr.shape[:2]
            x_min = max(0, x_min - padding)
            y_min = max(0, y_min - padding)
            x_max = min(w, x_max + padding)
            y_max = min(h, y_max + padding)

            cropped_face_bgr = original_image_bgr[y_min:y_max, x_min:x_max]

            if cropped_face_bgr.shape[0] > 0 and cropped_face_bgr.shape[1] > 0:
                cropped_face_rgb = cv2.cvtColor(cropped_face_bgr, cv2.COLOR_BGR2RGB)
                cropped_face_pil = Image.fromarray(cropped_face_rgb)
                output_crop_path = os.path.join(output_crop_dir, f"{base_filename}_face_{i}.jpg")

                cropped_face_pil.save(output_crop_path)
                print(f"Saved cropped face {i} to: {output_crop_path}")
            else:
                print(f"Warning: Skipped saving face {i} due to invalid dimensions after cropping.")
    else:
        print("No faces were detected in the image.")

Image loaded successfully: /home/pooh/Downloads/IMG_20250418_233052.jpg

0: 640x384 1 FACE, 29.2ms
Speed: 2.2ms preprocess, 29.2ms inference, 75.2ms postprocess per image at shape (1, 3, 640, 384)
Found 1 face(s). Cropping and saving...
Saved cropped face 0 to: cropped_faces/IMG_20250418_233052_face_0.jpg
